# Кредитный скоринг
При принятии решения о выдаче кредита или займа учитывается т.н. «Кредитный скоринг» — рейтинг платежеспособности клиента. ИИ на основе модели, которую просчитывает машинное обучение — в ней много параметров — возраст, зарплата, кредитная история, наличие недвижимости, автомобиля, судимости и других признаков, после обработки которых выносится положительное или отрицательное решение

Задание: ниже написан код на Pandas, написать аналогичный, но на Spark

In [1]:
# Импортируем библиотеки

import requests
import warnings
warnings.filterwarnings('ignore')

import pandas as pd   

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Данные:
[скачать](https://drive.google.com/file/d/1MuAyZiIm3b_r-AgQSj78tsRPqZpvv_2s/view?usp=sharing)

**application_record.csv**
*   ```ID```	- id клиента	
*   ```CODE_GENDER``` -	пол	
*   ```FLAG_OWN_CAR``` - есть ли машина	
*   ```FLAG_OWN_REALTY``` - есть ли недвижимость	
*   ```CNT_CHILDREN``` - количество детей	
*   ```AMT_INCOME_TOTAL``` - годовой доход	
*   ```NAME_INCOME_TYPE``` - категория дохода	
*   ```NAME_EDUCATION_TYPE```	- уровень образования	
*   ```NAME_FAMILY_STATUS``` - семейное положение	
*   ```NAME_HOUSING_TYPE```	- стиль жизни	
*   ```DAYS_BIRTH``` - день рождения Считается обратно от текущего дня (0), -1 означает вчера
*   ```DAYS_EMPLOYED```	- дата начала работы Считается обратно от текущего дня (0). Если значение положительное, значит, человек в настоящее время безработный
*   ```FLAG_MOBIL```	- есть ли мобильный	телефон
*   ```FLAG_WORK_PHONE```	- есть ли рабочий телефон	
*   ```FLAG_PHONE```	- есть ли стационарный телефон	
*   ```FLAG_EMAIL```	- есть ли электронная почта
*   ```OCCUPATION_TYPE```	- род занятий	
*   ```CNT_FAM_MEMBERS```	- размер семьи	

**credit_record.csv**
*   ```ID```	- id клиента
*   ```MONTHS_BALANCE```	месяц записи. Месяц извлеченных данных является отправной точкой, считая обратно, 0 - текущий месяц, -1 - предыдущий месяц и т.д.
*   ```STATUS```	- статус:
   *   ```0```: от 1 до 29 дней просрочки
   *   ```1```: от 30 до 59 дней просрочки
   *   ```2```: от 60 до 89 дней просрочки
   *   ```3```: от 90 до 119 дней просрочки
   *   ```4```: от 120 до 149 дней просрочки
    *   ```5```: просрочка или плохие долги, списанные более чем на 150 дней
    *   ```C```: выплачено в этом месяце X: нет кредита в этом месяце


## Считываем данные

In [2]:
# Ниже, мы для тех, у кого хоть раз были просрчоки больше 60 дней, ставим в таргет 1.
train_response = requests.get('https://drive.google.com/uc?id=1pKV_N_dgbnYSTNjb5SU7FhX4MJXUJLNF')
with open('application_record.csv', 'wb') as file:
    file.write(train_response.content)
data = pd.read_csv("application_record.csv", encoding = 'utf-8')

test_response = requests.get('https://drive.google.com/uc?id=17sFPpluSD_xvolqmAK0NZUHKGbAc81YC')
with open('credit_record.csv', 'wb') as file:
    file.write(test_response.content)
record = pd.read_csv("credit_record.csv", encoding = 'utf-8')


# # Добавляем срок кредита к параметрам выдачи кредита
begin_month = pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min) * - 1)
begin_month = begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'}) 
new_data = pd.merge(data, begin_month, how="left", on="ID") 

# # Больше 60, то это просрочка, ставим - Yes, если просрочка есть за срок кредита,то так же ставим Yes
record['dep_value'] = None
record['dep_value'][record['STATUS'] == '2'] = 'Yes'
record['dep_value'][record['STATUS'] == '3'] = 'Yes'
record['dep_value'][record['STATUS'] == '4'] = 'Yes'
record['dep_value'][record['STATUS'] == '5'] = 'Yes'
cpunt = record.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0] = 'Yes' 
cpunt['dep_value'][cpunt['dep_value'] == 0] = 'No'

# # Джойним всё вместе,заменяем Yes и No на 1 и 0
cpunt = cpunt[['dep_value']]
new_data = pd.merge(new_data, cpunt, how='inner', on='ID')
new_data['target'] = new_data['dep_value']
new_data.loc[new_data['target'] == 'Yes', 'target'] = 1
new_data.loc[new_data['target'] == 'No', 'target'] = 0

In [3]:
#  В итоге к анкетным данным мы добавили таргет
new_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,begin_month,dep_value,target
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,15.0,No,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,14.0,No,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,-1134,1,0,0,0,Security staff,2.0,29.0,No,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,4.0,No,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,26.0,No,0


In [4]:
# Упростим себе задачу и оставим только часть признаков
features = ['AMT_INCOME_TOTAL', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN']	
target = ['target',]
dataset = new_data[features + target]
dataset[target[0]] = pd.to_numeric(dataset[target[0]])

У нас есть выборка, где указаны параметры клиента, и вышел ли он на просрочку или нет.

In [5]:
# Разделим выборку на трейн и тест, на трейн будем обучать модель, на тест валидировать.
X_train, X_test, y_train, y_test = train_test_split(dataset[features], pd.to_numeric(dataset[target[0]]), test_size=0.3, random_state=42)

In [6]:
# Превращаем категориальные факторы в численные
ohe = OneHotEncoder()
ohe.fit(X_train[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
X_train_ohe = ohe.transform(X_train[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
X_test_ohe = ohe.transform(X_test[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])

X_train_ohe = pd.DataFrame(X_train_ohe.toarray(), columns=[item for sublist in ohe.categories_ for item in sublist])
X_test_ohe = pd.DataFrame(X_test_ohe.toarray(), columns=[item for sublist in ohe.categories_ for item in sublist])

In [7]:
# Отскалируем численные
mms = MinMaxScaler()
mms.fit(X_train[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])
X_train_scaled = mms.transform(X_train[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])
X_test_scaled = mms.transform(X_test[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])

X_train_scaled = pd.DataFrame(X_train_scaled, columns=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'])

In [8]:
X_train = pd.concat([X_train_scaled, X_train_ohe,], axis=1)
X_test = pd.concat([X_test_scaled, X_test_ohe, ], axis=1)

#  Модель

In [9]:
# Создадим простейшую модель, которая покажет через линейные коэффиценты связь переменных и таргета
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [10]:
train_score, test_score = accuracy_score(model.predict(X_train), y_train), accuracy_score(model.predict(X_test), y_test)
print(f'Точность модели на трейне {train_score}, на тесте {test_score}')

Точность модели на трейне 0.9828755045260394, на тесте 0.983635033827025


_____
_____

Аналогичный код на PySpark:

In [11]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=4e22a37b412174a48368e22b2e8bee9103622aba9204509e26e9075715794c1c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [12]:
# зададим импорты
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml import Pipeline

In [13]:
# создадим сессия
spark = SparkSession.builder\
        .master('local[2]')\
        .appName('Lesson_7')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '4g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

In [14]:
# Загрузим данные
data = spark.read.csv('application_record.csv', header=True, inferSchema=True)
record = spark.read.csv('credit_record.csv', header=True, inferSchema=True)

# Добавим столбец begin_month
begin_month = record.groupby('ID').agg(F.min(F.col('MONTHS_BALANCE')).alias('begin_month'))
begin_month = begin_month.withColumn('begin_month', -1 * F.col('begin_month'))
new_data = data.join(begin_month, on='ID', how='left')

# Добавим столбец target
record = record.withColumn('dep_value', 
                           F.when(F.col('STATUS').isin(['2', '3', '4', '5']), 'Yes').otherwise(None))
cpunt = record.groupby('ID').count().withColumnRenamed('count', 'dep_value')
cpunt = cpunt.withColumn('dep_value', F.when(F.col('dep_value') > 0, 'Yes').otherwise('No'))
new_data = new_data.join(cpunt.select('ID', 'dep_value'), on='ID', how='inner')
new_data = new_data.withColumn('target', F.when(F.col('dep_value') == 'Yes', 1).otherwise(0))

посмотрим на схему и вид заполнения:

In [15]:
new_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)
 |-- begin_month: integer (nullable = true)
 |-- dep_value: string (nullable = false)
 |-- target: integer (nullable = false)



In [16]:
new_data.show(1, vertical=True)

-RECORD 0-------------------------------
 ID                  | 5008804          
 CODE_GENDER         | M                
 FLAG_OWN_CAR        | Y                
 FLAG_OWN_REALTY     | Y                
 CNT_CHILDREN        | 0                
 AMT_INCOME_TOTAL    | 427500.0         
 NAME_INCOME_TYPE    | Working          
 NAME_EDUCATION_TYPE | Higher education 
 NAME_FAMILY_STATUS  | Civil marriage   
 NAME_HOUSING_TYPE   | Rented apartment 
 DAYS_BIRTH          | -12005           
 DAYS_EMPLOYED       | -4542            
 FLAG_MOBIL          | 1                
 FLAG_WORK_PHONE     | 1                
 FLAG_PHONE          | 0                
 FLAG_EMAIL          | 0                
 OCCUPATION_TYPE     | null             
 CNT_FAM_MEMBERS     | 2.0              
 begin_month         | 15               
 dep_value           | Yes              
 target              | 1                
only showing top 1 row



In [17]:
# Упростим себе задачу и оставим только часть признаков
features = ['AMT_INCOME_TOTAL', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'target']
dataset = new_data.select(features)
dataset = dataset.withColumn('target', F.col('target').cast('double'))

In [18]:
# Разделим выборку на трейн и тест, на трейн будем обучать модель, на тест валидировать.
X_train, X_test = dataset.randomSplit([0.7, 0.3], seed=42)

In [19]:
# Создаем преобразование категориальных переменных в числовые
stringIndexer = StringIndexer(inputCols=['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY'], 
                              outputCols=['CODE_GENDER_Index', 'FLAG_OWN_CAR_Index', 'FLAG_OWN_REALTY_Index'])

ohe = OneHotEncoder(inputCols=['CODE_GENDER_Index', 'FLAG_OWN_CAR_Index', 'FLAG_OWN_REALTY_Index'], 
                    outputCols=['CODE_GENDER_OHE', 'FLAG_OWN_CAR_OHE', 'FLAG_OWN_REALTY_OHE'])

In [20]:
# Создаем преобразование числовых переменных
assembler_numerical = VectorAssembler(inputCols=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'], outputCol='numerical_features')
mms = MinMaxScaler(inputCol='numerical_features', outputCol='scaled_numerical_features')

In [21]:
# Создаем преобразование всех признаков в единый вектор
assembler_all = VectorAssembler(inputCols=['AMT_INCOME_TOTAL', 'CNT_CHILDREN', 'CODE_GENDER_OHE', 
                                            'FLAG_OWN_CAR_OHE', 'FLAG_OWN_REALTY_OHE'], outputCol='features')

In [22]:
# Создаем модель логистической регрессии
lr = LogisticRegression(featuresCol='features', labelCol='target')

In [23]:
# Создаем pipeline
pipeline = Pipeline(stages=[stringIndexer, ohe, assembler_numerical, mms, assembler_all, lr])

In [24]:
# Обучаем модель
model = pipeline.fit(X_train)

# Получаем прогнозы
train_predictions = model.transform(X_train)
test_predictions = model.transform(X_test)

# Оцениваем качество модели
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', 
                                              labelCol='target', 
                                              metricName='accuracy')
train_score = evaluator.evaluate(train_predictions)
test_score = evaluator.evaluate(test_predictions)
print(f'Точность модели на трейне {train_score}, на тесте {test_score}')

Точность модели на трейне 1.0, на тесте 1.0


Полученный результат может показаться странным, но это связано с различиями в реализации алгоритма логистической регрессии в библиотеках Pandas и PySpark.

В частности, в библиотеке Pandas для решения задачи бинарной классификации методом логистической регрессии используется оптимизационный алгоритм L-BFGS, который не гарантирует сходимость к глобальному оптимуму. В то же время, в библиотеке PySpark для решения этой же задачи используется стохастический градиентный спуск (SGD), который имеет более стабильную сходимость к глобальному оптимуму.

Поэтому различия в точности модели между Pandas и PySpark могут быть связаны с использованием различных алгоритмов оптимизации. 

Но вообщемы видим что в Pandas что в Spark обень высокую точность на тестовой выборке, что может быть признаком переобучения. Стоило бы ещё провести дополнительный анализ для проверки корректности работы модели перед тем, как всерьёз полагаться на неё. Но у нас учебнывй пример на понимание и использование синтаксиса, так что ограничимся тем, что на сей занимательный факт мы обратили внимание.